In [ ]:

## Setting constant GLOBALS for all runs
#### DESIGN RUN NAME #####################################################################################################
DESIGN_FOLDER     = ''   # Variable
FOLDER_MATCH      = 'MATCH' #Not needed for now -> This is just for match
FOLDER_PARENT     = '../Parent_1ohp' # -> Adjust, contains ONLY parent pdbs
##########################################################################################################################
#### Input parameters ####################################################################################################
MAX_JOBS          = 100
N_PARENT_JOBS     = 200
MAX_DESIGNS       = 4000
KBT_BOLTZMANN     = 0.15    # Variable
CST_WEIGHT        = 0    # Variable
HIGHSCORE         = 0.70
NEG_BEST          = 0.59
ProteinMPNN_PROB  = 0.0
PMPNN_BIAS        = 0.0
LMPNN_PROB        = 0.0
WT                = "1ohp"
LIGAND            = "5TS"
REPACK            = "36,45,51,67"
MATCH             = "14,18,26,30,55,65,80,82,99,101,112"
DESIGN            = "7,10,11,14,15,18,26,29,30,38,54,55,58,59,63,65,71,73,78"
DESIGN           += ",80,82,84,86,93,95,97,99,101,103,109,112,114,116,121"
ProteinMPNN_T     = "0.1"
LMPNN_T           = "0.1"
SUBMIT_PREFIX     = ""   # Variable
BLUEPEBBLE        = False
GRID              = True
BACKGROUND_JOB    = False
ABBIE_LOCAL       = False

if GRID:            ROSETTA_PATH = "/home/bunzelh/rosetta_src_2021.16.61629_bundle/main/source/"      #GRID
if BLUEPEBBLE:      
                    ROSETTA_PATH = "/sw/apps/rosetta/rosetta_src_2021.16.61629_bundle/main/source/"   #BLUEPEBBLE 
                    BLUEPEBBLE_ACCOUNT = "ptch000361"
                    #BLUEPEBBLE_ACCOUNT = "ptch000721"
if BACKGROUND_JOB:  ROSETTA_PATH = "/sw/apps/rosetta/rosetta_src_2021.16.61629_bundle/main/source/"   #BLUEPEBBLE 
    
NORM              = {'interface_score': [10, 35], 'total_score': [200, 500], 'catalytic_score': [-40, 0], 'efield_score': [10, 220]}
##########################################################################################################################
AIZYME_TOOLS      = "/home/lmerlicek/AIzymes/src/AIzymes_011.ipynb"
FIELD_TOOLS       = "/home/lmerlicek/AIzymes/src/FieldTools.py"
%run "$AIZYME_TOOLS"

In [ ]:
# Define the ranges for your hyperparameters
CST_WEIGHT_values = [1] # , 2, 3, 4
kbt_decay_values = [0.001] # , 0.0005, 0.0003
#To avoid inf values in the selection probabilities the lowest kbt should be around 0.01.
#For 4000 MAX_DESIGNS that corresponds to a decay value of around 0.001

# Function to run the controller with given parameters
def run_controller(cst, kbt_decay, home_dir):
    global CST_WEIGHT, DESIGN_FOLDER, SUBMIT_PREFIX, KBT_BOLTZMANN
    # Set the parameters in your environment or pass them to your application
    CST_WEIGHT = cst
    KBT_BOLTZMANN = [0.5, kbt_decay]
    DESIGN_FOLDER = home_dir
    SUBMIT_PREFIX = f"BD_{kbt_decay}"

    print(f"Running pos control with CST: {cst}")
    %run "$AIZYME_TOOLS"
    #run controller with PROMT = Flase to avoid y/n promt
    controller(RESET=False, EXPLORE=False, PROMPT=False, UNBLOCK_ALL=False, PRINT_VAR=True, PLOT_DATA=True)
    get_best_structures()

    

# Iterate over all combinations of the hyperparameters
for cst in CST_WEIGHT_values:
    for kbt_decay in kbt_decay_values:
        # Generate a unique home directory name for this combination
        home_dir = f"LM_CNTRpos_4mvr_B_decay_{kbt_decay}_CST_{cst}_011".replace('.', '')
        
        # Run the controller with the current combination of parameters
        run_controller(cst, kbt_decay, home_dir)
    
